In [1]:
from flask import Flask, request, Response, send_file
import jsonpickle
import numpy as np
import cv2
from PIL import Image
from io import StringIO
import matplotlib.pyplot as plt

from torchvision.models import resnet18
import torch.nn as nn

from torchvision.transforms import ToTensor, Resize, Compose
import torch
# Initialize the Flask application
app = Flask(__name__)

model = resnet18(True)

layer = model.layer1

In [25]:
layer = model.layer1

cat = Image.open("cat.jpg")

input = Compose([Resize((224,224)), ToTensor()])(cat)

input = input.view(1,3,224,224)

output = model(input)


def traverse(module):
    try:
        module.child
    except AttributeError:
        module.child = []
        
    for m in module.children():
        if m not in module.child:
            module.child.append(m)
            traverse(m)

traverse(model)
print(model.child[4].child)

[BasicBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
), BasicBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)]


In [ ]:
def make_graph(var):
    seen = set()

    def add_nodes(var):
        if var not in seen:
            if isinstance(var, Variable):
                value = '('+(', ').join(['%d'% v for v in var.size()])+')'
                dot.node(str(id(var)), str(value), fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'previous_functions'):
                for u in var.previous_functions:
                    dot.edge(str(id(u[0])), str(id(var)))
                    add_nodes(u[0])
    add_nodes(var.creator)
    return dot

In [ ]:
x = input
for l in model.children():
    print(l)
    x = l(x)
    out = x.squeeze()
    print(out.shape)

    f, w, h = out.shape

    for img in out:
        img = img.detach().numpy()
        plt.title(l)
        plt.imshow(img)
        plt.show()